# Data Exploration
## Purpose
State the purpose of the notebook.
## Methodology
Quickly describle assumptions and processing steps.
## WIP - improvements
Use this section only if the notebook is not final.

Notable TODOs:

- Todo 1;
- Todo 2;

## Results
Describe and comment the most important results.

# Setup
## Library import
We import all the required Python libraries

In [83]:
import os
import fiona
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray
import rasterio
import regionmask
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import box
from tqdm import tqdm

## Utils

**set_lat_lon_attrs**

In [ ]:
def set_lat_lon_attrs(ds):
    """ Set CF latitude and longitude attributes"""
    ds["lon"] = ds.lon.assign_attrs({
      'axis' : 'X',
       'long_name' : 'longitude',
        'standard_name' : 'longitude',
         'stored_direction' : 'increasing',
          'type' : 'double',
           'units' : 'degrees_east',
            'valid_max' : 360.0,
             'valid_min' : -180.0
             })
    ds["lat"] = ds.lat.assign_attrs({
      'axis' : 'Y',
       'long_name' : 'latitude',
        'standard_name' : 'latitude',
         'stored_direction' : 'increasing',
          'type' : 'double',
           'units' : 'degrees_north',
            'valid_max' : 90.0,
             'valid_min' : -90.0
             })
    return ds

**create_ds_mask**

In [ ]:
def create_ds_mask(df, ds, name, lon_name='lon', lat_name='lat'):
    """Create masks of geographical regions"""
    # Create index column
    if 'index' not in df:
        df = df.reset_index(drop=True).reset_index()

    # Extract indexes and geoms that are large enough!
    id_ints = df['index'].values
    geoms = df['geometry'].values
    
    print(f'Number of indexes: {len(id_ints)}')
    print(f'Number of geoms: {len(geoms)}')


    # create mask object
    da_mask = regionmask.Regions(
      name = name,
      numbers = id_ints,
      outlines = geoms)\
      .mask(ds, lon_name=lon_name, lat_name=lat_name)\
      .rename(name)

    # get the ints actually written to mask
    id_ints_mask = da_mask.to_dataframe().dropna()[name].unique()
    id_ints_mask = np.sort(id_ints_mask).astype('int')
    
    print(f'Number of ints in mask: {len(id_ints_mask)}')
    
    # get the ints not written to mask
    id_ints_not_in_mask = df[~df['index'].isin(id_ints_mask)]['index'].values
    
    if len(id_ints_not_in_mask) > 0: 
        print(f'Ints not in mask: {id_ints_not_in_mask}')
    
    # update da attributes
    da_mask.attrs['id_ints'] = id_ints_mask
    da_mask = set_lat_lon_attrs(da_mask)
    
    return da_mask, id_ints_not_in_mask

**find_nearest**

In [ ]:
def find_nearest(array, value):
    """Find nearest value in numpy array"""
    array = np.asarray(array)
    
    # Get the mean step values
    step = np.abs(np.diff(array)).max()
    
    # Find the nearest values
    diff = np.abs(array - value)
    idx = np.argwhere((diff >= np.amin(diff) - step) & (diff <= np.amin(diff) + step))

    return idx

**get_xy_from_latlon**

In [ ]:
def get_xy_from_latlon(ds, lat, lon):
    """Return the x/y values for a given longitude/latitude values"""
    # Read all lon/lat values
    lons = ds.lon.data
    lats = ds.lat.data
    
    # Find the positions of the nearest longitude/latitude values
    idx_lon = find_nearest(lons, lon)
    idx_lat = find_nearest(lats, lat)
    
    # Check the identical rows in both arrays
    res = (idx_lon[:, None] == idx_lat).all(-1).any(-1)
    yx_positions = idx_lon[res]
    
    
    if yx_positions.shape[0] == 0:
        raise Exception("Sorry, lat/lon values outside data domain")   
    if yx_positions.shape[0] > 1:
        # If more than one identical rows take the row nearest to the mean value
        yx_positions = np.mean(yx_positions,axis=0).astype(int).reshape(1,2)

    # Get the x/y values
    x_position = yx_positions[0][1]
    y_position = yx_positions[0][0]
    x = ds.rlon.data[yx_positions[0][1]]
    y = ds.rlat.data[yx_positions[0][0]]

    return x_position, y_position, x, y

# Data import

## Comarcas Agrarias
**[Data source](https://www.mapa.gob.es/es/cartografia-y-sig/ide/descargas/agricultura/)**

La distribución de la superficie de España en `Comarcas Agrarias` agrupa los territorios en unidades espaciales intermedias entre la provincia y el municipio que sin personalidad jurídico-administrativa alguna, tiene un carácter uniforme desde el punto de vista agrario.

In [ ]:
comarcas_agr = gpd.read_file(f'../../datasets/raw/georegions/ComarcasAgrarias/ComarcasAgrarias.shp')
comarcas_agr.sort_values(['CO_CCAA', 'CO_PROVINC', 'CO_COMARCA'], inplace = True)

Remove Canarias, Ceuta, and Melilla

In [ ]:
comarcas_agr = comarcas_agr[~comarcas_agr['DS_CCAA'].isin(['Canarias', 'Ceuta', 'Melilla'])]
comarcas_agr = comarcas_agr.reset_index(drop=True)

**Display geometries**

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
comarcas_agr.plot(ax=ax, color='w', edgecolor='k')

## Municipios de España

**[Data source](https://opendata.esri.es/datasets/53229f5912e04f1ba6dddb70a5abeb72_0/explore?location=43.017075%2C9.288571%2C5.20)**

In [ ]:
municipios = gpd.read_file(f'../../datasets/raw/georegions/Municipios/Municipios_IGN.shp')
municipios.sort_values(['CODNUT1', 'CODNUT2', 'CODNUT3', 'CODIGOINE'], inplace = True)

Remove Canarias, Ceuta, and Melilla

In [ ]:
municipios = municipios[~municipios['CODNUT2'].isin(['ES70', 'ES63', 'ES64'])]
municipios = municipios.reset_index(drop=True)

**Display geometries**

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
municipios.plot(ax=ax, color='w', edgecolor='k')

## Fire danger indicators for Europe 
**[Data source](https://cds.climate.copernicus.eu/cdsapp#!/dataset/sis-tourism-fire-danger-indicators?tab=overview)**

The dataset presents projections of fire danger indicators for Europe based upon the Canadian Fire Weather Index System (FWI) under future climate conditions. The FWI is a meteorologically based index used worldwide to estimate the fire danger and is implemented in the Global ECMWF Fire Forecasting model (GEFF).

**Variables:**
- **Seasonal fire weather index:** 
The mean fire weather index value over the European fire season (June-September). This is calculated as the sum of the daily fire weather index over the European fire season divided by the total number of days within this date range. The higher the index value, the more favorable the meteorological conditions to trigger a wildfire are.

**Read data**

In [ ]:
data_dir = '../../datasets/raw/climate/dataset-sis-tourism-fire-danger-indicators/'
for n, file in enumerate(os.listdir(data_dir)[16:]):
    # convert to Dataset and concatenate by time
    if n == 0:
        ds_fire = xr.open_dataset(data_dir+file, engine="netcdf4")
    else:
        ds = xr.open_dataset(data_dir+file, engine="netcdf4")
        ds_fire = xr.concat([ds_fire, ds], dim='time')

In [ ]:
ds_fire

**Multidimensional Coordinates**

The data will derive from a numerical model in which the poles of the model's coordinate system (`logical coordinates`) differ from the earth's true poles (`physical coordinates`). This is typically done when running limited area models, in order to keep the poles as far away as possible from the area that is being modelled. This allows the model's resolution to be roughly uniform over the model domain, as the coordinate system is then approximately cartesian and avoids issues where the meridians converge close to the poles.

In our dataset, the `logical coordinates` are `rlon` and `rlat`, while the physical coordinates are `lon` and `lat`, which represent the latitudes and longitude of the data.

In [ ]:
print(ds_fire.rlon.attrs)
print(ds_fire.rlat.attrs)

In [ ]:
print(ds_fire.lon.attrs)
print(ds_fire.lat.attrs)

**Display data**

Xarray provides [several ways](http://xarray.pydata.org/en/stable/examples/multidimensional-coords.html) to plot and analyze such datasets.

If we try to plot the data variable `fwi-mean-jjas`, by default we get the logical coordinates.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20,10))
ds_fire['fwi-mean-jjas'].isel(time=0).plot(ax=axs[0])
ds_fire['fwi-mean-jjas'].isel(time=-1).plot(ax=axs[1])

In order to visualize the data on a conventional latitude-longitude grid, we can take advantage of xarray’s ability to apply [cartopy](http://scitools.org.uk/cartopy/index.html) map projections.

In [ ]:
plt.figure(figsize=(10,10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ds_fire['fwi-mean-jjas'].isel(time=0).plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), x='lon', y='lat', add_colorbar=False)
ax.coastlines()
ax.set_ylim([0,90]);
ax.set_xlim([-30,70]);

## Rasterize vector data

### Compute mean values for each `comarca agraria`

**Create the data mask by rasterizing the vector data**

In [ ]:
gdf = comarcas_agr.copy()
gdf = gdf.reset_index(drop=True)
da_mask, id_ints_not_in_mask = create_ds_mask(gdf, ds_fire, name='mask', lon_name='lon', lat_name='lat')

In [ ]:
da_mask

**Add geometries smaller than mean cell size into the mask**

In [ ]:
gdf_not_in_mask = gdf.iloc[id_ints_not_in_mask].copy()

gdf_not_in_mask['centroid'] = gdf_not_in_mask['geometry'].apply(lambda x: x.centroid)

for id_int in id_ints_not_in_mask:
    lon = gdf_not_in_mask['centroid'].loc[id_int].x
    lat = gdf_not_in_mask['centroid'].loc[id_int].y
    
    # Get x/y values for the corresponding longitude/latitude values
    x_pos, y_pos, x, y = get_xy_from_latlon(ds_fire, lat, lon)
    
    # Replace cell value with new int
    da_mask.data[y_pos, x_pos] = id_int
    
# update da attributes
da_mask.attrs['id_ints'] = list(gdf.index)

**Display mask**

In [ ]:
plt.figure(figsize=(10,10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
da_mask.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), x='lon', y='lat', add_colorbar=False)
ax.coastlines()
ax.set_ylim([35, 45]);
ax.set_xlim([-10, 5]);

**Add mask as a new variable into the xarray.Dataset**

In [ ]:
ds_fire['mask'] = da_mask

**Compute mean value over time**

In [ ]:
mean_values = [] 
for index in gdf.index:
    mean_values.append(ds_fire['fwi-mean-jjas'].where(ds_fire.mask == index).mean(['rlon', 'rlat']).values)
    
gdf['fire'] = mean_values
gdf['time'] = [list(ds_fire.coords['time'].values)]*len(mean_values)

**Display values for a single year**

In [ ]:
year = 0
gdf_1year = gdf.copy()
gdf_1year['fire'] = gdf_1year['fire'].apply(lambda x: x[year])
gdf_1year['time'] = gdf_1year['time'].apply(lambda x: x[year])

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

divider = make_axes_locatable(ax)

cax = divider.append_axes("right", size="5%", pad=-0.5)

gdf_1year.plot(ax=ax, column='fire', cmap='magma', legend=True, cax=cax, legend_kwds={'label': "Seasonal fire weather index"})

ax.set_title(str(gdf_1year['time'].iloc[0]))

## Mapas de presencia de especies forestales en España peninsular

In [ ]:
xda = xr.open_rasterio(f'../../datasets/raw/especies_forestales/g724_mfe_pres/quer_sube.tif').squeeze().drop("band")

In [ ]:
xda

In [ ]:
# convert to Dataset
xds = xr.Dataset({'quer_sube': xda}, attrs=xda.attrs)

In [ ]:
xds

In [ ]:
xds.quer_sube.plot()

In [ ]:
xr.combine_by_coords([ds_fire, xds])